In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,1769
2,Huelva,Confirmados PDIA 14 días,524
3,Huelva,Tasa PDIA 14 días,"102,59563028763978"
4,Huelva,Confirmados PDIA 7 días,251
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,9
629,Municipio de Huelva sin especificar,Total Confirmados,32
630,Municipio de Huelva sin especificar,Curados,1


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1769.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  432.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2113 personas en los últimos 7 días 

Un positivo PCR cada 1381 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,1769.0,251.0,524.0,510743.0,49.144090,102.595630,55.0
Huelva-Costa,883.0,116.0,208.0,288115.0,40.261701,72.193395,31.0
Huelva (capital),432.0,68.0,104.0,143663.0,47.332995,72.391639,21.0
Condado-Campiña,621.0,105.0,245.0,155057.0,67.717033,158.006411,9.0
Sierra de Huelva-Andévalo Central,233.0,21.0,60.0,67571.0,31.078421,88.795489,9.0
Villanueva de los Castillejos,16.0,9.0,14.0,2820.0,319.148936,496.453901,4.0
Jabugo,11.0,6.0,7.0,2250.0,266.666667,311.111111,3.0
Aljaraque,75.0,4.0,8.0,21260.0,18.814675,37.629351,2.0
Almonte,60.0,6.0,8.0,24191.0,24.802613,33.070150,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campillo (El),27.0,1.0,25.0,2023.0,49.431537,1235.788433,NaN
Manzanilla,19.0,3.0,14.0,2135.0,140.515222,655.737705,0.0
Villanueva de los Castillejos,16.0,9.0,14.0,2820.0,319.148936,496.453901,4.0
San Juan del Puerto,66.0,30.0,45.0,9300.0,322.580645,483.870968,2.0
Villarrasa,14.0,6.0,10.0,2176.0,275.735294,459.558824,1.0
Palma del Condado (La),87.0,16.0,49.0,10761.0,148.685066,455.348016,1.0
Bollullos Par del Condado,106.0,18.0,49.0,14272.0,126.121076,343.329596,0.0
Jabugo,11.0,6.0,7.0,2250.0,266.666667,311.111111,3.0
Lucena del Puerto,14.0,3.0,9.0,3371.0,88.994364,266.983091,1.0
